# Imports

In [65]:
import glob
import functools
import os
import gzip
import pandas as pd

import matplotlib
%matplotlib inline
matplotlib.rcParams['figure.figsize'] = (20, 10)

In [83]:
def for_d3(file):
    idx = pd.Series(0, index=pd.date_range('2016-01-01', '2016-09-01', freq='H'), name='all_ts')\
        .asfreq('30Min', method='pad')
    df = pd.read_csv(gzip.open(file), delimiter=',')\
        .assign(minute=lambda df: df.minute * 30)\
        .assign(time=lambda df: df.date.astype(str) + ' ' + df.hour.astype(str).str.zfill(2) + ':' + df.minute.astype(str).str.zfill(2))\
        [['time', 'dropoffs']]\
        .set_index('time')
    return df\
        .join(idx, how='right')\
        .fillna(0)\
        [lambda df2: df2.index <= df.index.max()]\
        .reset_index()\
        .rename(columns={'index': 'time'})\
        .assign(dropoffs=lambda df: df.dropoffs.astype(int))\
        .drop('all_ts', axis=1)

In [87]:
for f in glob.glob(os.path.join('data', 'dropoffs*0.003.csv.gz')):
    for_d3(f).to_csv(os.path.join('for_d3', os.path.basename(f)[:-3]), index=False)

In [84]:
for_d3(os.path.join('data', 'dropoffs_bowery_ballroom_0.003.csv.gz')).head()

,time,dropoffs
0,2016-01-01 00:00:00,72
1,2016-01-01 00:30:00,94
2,2016-01-01 01:00:00,106
3,2016-01-01 01:30:00,109
4,2016-01-01 02:00:00,90
